# Test Full-text Search Query

In [50]:
import os
import json
import pandas as pd
import qgrid
import tqdm

#widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

ES_URL = 'localhost:9200/'

def curl_es(q):
    try:
        f = os.popen(q)
        j = json.load(f)
        f.close()
    except Exception as e:
        print(e,f.readlines())
        f.close()
        return 'cURL failed'
    return j

def gen_test_results(search_terms):
    ds = []
    for search_term in tqdm.tqdm_notebook(search_terms):
        query = f'curl -X GET "{ES_URL}tops_sample/_search"' + \
            " -H 'Content-Type: application/json' -d' " +\
            query_pattern.replace('xxsearch_term', search_term) + "'"
        j = curl_es(query)
        try:
            skus = j['hits']['hits']
        except:
            continue
        for i in range(len(skus)):
            try:
                name_en = skus[i]['_source']['name_en']
            except:
                name_en = 'invalid_name_en'
            try:
                brand_name_en = skus[i]['_source']['brand_en']
            except:
                brand_name_en = 'invalid_brand_name'
            try:
                categories_en = skus[i]['_source']['class_en']
            except:
                categories_en = 'invalid_categories'
            d = {'search_term':search_term,'position':i,'name_en':name_en,'brand_name_en':brand_name_en,
                 'categories_en':categories_en,'_score':skus[i]['_score']}
            ds.append(d)
    return pd.DataFrame(ds)

def display_results(df, search_term):
    return qgrid.show_grid(df[df.search_term==search_term].iloc[:,1:])


with open('data/tops_sample_query.json','r') as f: query_pattern = f.read()

In [51]:
#enter your search terms to test here
search_terms = [
'eggs','fresh eggs',
'milk','fresh milk'
]
res = gen_test_results(search_terms)

/Users/admin/.virtualenvs/sandbox/lib/python3.7/site-packages/ipykernel_launcher.py:27: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


## See Results

In [52]:
interact(display_results, df=fixed(res), search_term = widgets.Dropdown(options=search_terms))

interactive(children=(Dropdown(description='search_term', options=('eggs', 'fresh eggs', 'milk', 'fresh milk')…

<function __main__.display_results(df, search_term)>